In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
from helper import *

In [2]:
data_folder = 'dataset/'
train_transaction = pd.read_csv(data_folder+'train_transaction.csv')
train_identity = pd.read_csv(data_folder+'train_identity.csv')
test_transaction = pd.read_csv(data_folder+'test_transaction.csv')
test_identity = pd.read_csv(data_folder+'test_identity.csv')
categories_list = ['id_'+str(x) for x in range(12,39)]+['DeviceInfo','DeviceType','R_emaildomain','P_emaildomain','addr1','addr2','ProductCD'] + ['M'+str(x) for x in range(1,10)] + ['card'+str(x) for x in range(1,7)]

In [3]:
train = reduce_memory_usage(pd.merge(train_transaction, train_identity, on='TransactionID', how='left'),categories=categories_list)
test = reduce_memory_usage(pd.merge(test_transaction, test_identity, on='TransactionID', how='left'),categories=categories_list)

Mem. usage decreased to 524.76 Mb (73.2% reduction)
Mem. usage decreased to 457.64 Mb (72.7% reduction)


In [4]:
del train_identity, train_transaction, test_identity, test_transaction

In [9]:
from helper import *
normalize(train['TransactionAmt'])

nan
nan


0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
590535   NaN
590536   NaN
590537   NaN
590538   NaN
590539   NaN
Name: TransactionAmt, Length: 590540, dtype: float16

In [15]:
summarize(train[['TransactionAmt']])

Dataset Shape: (590540, 1)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value,Entropy
0,TransactionAmt,float16,0,8195,68.5,29.0,59.0,8.1


In [34]:
train['dist1'].mean(skipna=True)

nan